In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [116]:
poblacion_teorica = pd.read_excel("../data/processed/proyeccion_problemas_de_salud.xlsx")

poblacion_teorica["Diagnostico"] = poblacion_teorica.Diagnostico.str.split().str[0]
poblacion_teorica = poblacion_teorica.query("Estrato == 'Pais'")

DIAGS_RELEVANTES = poblacion_teorica.Diagnostico.unique()
poblacion_teorica = poblacion_teorica.set_index("Diagnostico")
# Selecciona solamente la poblacion prevista
poblacion_teorica = poblacion_teorica[[i for i in range(2017, 2036)]]

In [101]:
egresos_int = pd.read_csv("../data/processed/egresos_torax_mes_y_dia.csv")
egresos_int = egresos_int[egresos_int["DIAG1"].isin(DIAGS_RELEVANTES)]
egresos_int = egresos_int.query("ANO_EGRESO >= 2017")

In [102]:
egresos_pais = pd.read_csv(
    "../data/processed/ranking_nacional_egresos.csv",
    sep=";",
    encoding="latin-1",
    usecols=[
        "ANO_EGRESO",
        "ESTABLECIMIENTO_SALUD",
        "DIAG1",
        "n_egresos",
        "dias_estada_totales",
        "n_int_q",
        "n_muertos",
    ],
)

egresos_pais = egresos_pais[egresos_pais["DIAG1"].isin(DIAGS_RELEVANTES)]
egresos_pais = egresos_pais.query("ANO_EGRESO >= 2017")

## Separación de población teórica en hospitalizados y ambulatorios

In [117]:
# De los casos totales, se asume que una parte (0.8) se resuelve en hospitalizacion
RATIO_HOSPITALIZADOS = 0.8
RATIO_AMBULATORIO = 1 - RATIO_HOSPITALIZADOS

poblacion_teorica_hospitalizados = poblacion_teorica * RATIO_HOSPITALIZADOS
poblacion_teorica_ambulatorio = poblacion_teorica * RATIO_AMBULATORIO

In [103]:
tabla_dinamica_egresos_pais = pd.pivot_table(
    egresos_pais,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos"],
    aggfunc="sum",
    fill_value=0,
)

tabla_dinamica_egresos_int = pd.pivot_table(
    egresos_int,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos"],
    aggfunc="sum",
    fill_value=0,
)

In [104]:
tabla_dinamica_egresos_pais.columns = tabla_dinamica_egresos_pais.columns.droplevel(0)
tabla_dinamica_egresos_int.columns = tabla_dinamica_egresos_int.columns.droplevel(0)

In [105]:
poblacion_teorica.merge(
    tabla_dinamica_egresos_pais,
    how="inner",
    left_index=True,
    right_index=True,
    suffixes=("_teoricos", "_egresos_pais"),
).round(0)

,Incidencia (Cada 100.000),Incidencia %,País Incidencia,Sexo Incidencia,Edad Incidencia,Estrato,2017_teoricos,2018_teoricos,2019_teoricos,2020_teoricos,...,2030,2031,2032,2033,2034,2035,2017_egresos_pais,2018_egresos_pais,2019_egresos_pais,2020_egresos_pais
C33X,0.0,0.0,Internacional,Ambos,Todas,Pais,18.0,19.0,19.0,19.0,...,21.0,21.0,21.0,21.0,21.0,21.0,25,18,17,14
C340,7.0,0.0,Chile,Ambos,Todas,Pais,1271.0,1294.0,1318.0,1343.0,...,1431.0,1437.0,1443.0,1449.0,1454.0,1459.0,163,143,148,103
C341,7.0,0.0,Chile,Ambos,Todas,Pais,1271.0,1294.0,1318.0,1343.0,...,1431.0,1437.0,1443.0,1449.0,1454.0,1459.0,372,478,493,316
C342,7.0,0.0,Chile,Ambos,Todas,Pais,1271.0,1294.0,1318.0,1343.0,...,1431.0,1437.0,1443.0,1449.0,1454.0,1459.0,50,68,52,48
C343,7.0,0.0,Chile,Ambos,Todas,Pais,1271.0,1294.0,1318.0,1343.0,...,1431.0,1437.0,1443.0,1449.0,1454.0,1459.0,258,293,279,223
C381,1.0,0.0,Chile,Ambos,Todas,Pais,221.0,225.0,229.0,233.0,...,249.0,250.0,251.0,252.0,253.0,254.0,41,41,52,42
C384,0.0,0.0,Internacional,Ambos,Todas,Pais,6.0,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,39,44,52,43
C450,0.0,0.0,Internacional,Ambos,Todas,Pais,6.0,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,81,76,75,53
C780,7.0,0.0,Chile,Ambos,Todas,Pais,1271.0,1294.0,1318.0,1343.0,...,1431.0,1437.0,1443.0,1449.0,1454.0,1459.0,373,379,430,326
C782,7.0,0.0,Chile,Ambos,Todas,Pais,1271.0,1294.0,1318.0,1343.0,...,1431.0,1437.0,1443.0,1449.0,1454.0,1459.0,147,188,211,200
